# Проект: исследование надежности заемщиков.

Главная цель проекта: найти факторы, которые влияют на погашение кредита в срок. В ходе выполнения проекта необходимо выяснить оказывают ли влияние на возвращение кредита  такие факторы, как: семейное положение, количеством детей, уровень дохода.


[click on this link](#my-multi-word-header)


[Chapter 1](#chapter1)

**Структура проекта:**
    
<a href='#1'><h5>1. Обзор данных </a>
<a href='#2'><h5>2. Заполнение пропусков </a>
<a href='#3'><h5>3. Проверка данных на аномалии и их исправление </a>
<a href='#4'><h5>4. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.</a>
<a href='#5'><h5>5. Категоризация дохода </a>
<a href='#6'><h5>6. Категоризация целей кредита </a>
<a href='#7'><h5>7. Выводы. Ответы на вопросы </a>

### 1. Обзор данных <a id='1'></a>

In [9]:
# загрузка библиотек
#
# работа с таблицами и табличными данными
import pandas as pd

# получение названия файла БД
import os

In [18]:
# загрузка БД с сервера или ПК
# attempt_1 - путь и название файла для загрузки с сервера
# attempt_2 - путь и название файла для загрузки с ПК

attempt_1 ='https://code.s3.yandex.net/datasets/data.csv'
attempt_2 ='C://yandex_data_sets/01_data.csv'

# берем название файла из указанного пути
name_os_data = os.path.basename(attempt_1)
print('File name:',name_os_data)

# обработка ошибок при загрузке
try:
    df = pd.read_csv(attempt_1)
    print(name_os_data, 'has been downloaded from the server')
except:
    print('Server error')
try:
    df = pd.read_csv(attempt_2)
    print(name_os_data, 'has been downloaded from local host')
except:
    print('Local Error')

File name: data.csv
data.csv has been downloaded from the server
data.csv has been downloaded from local host


<a class="anchor" id="chapter1"> тест</a>

In [3]:
# выводим заголовок
display(df.head(3))

 # общая информация о таблице
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из общей информации видно, что в столбцах days_employed (трудовой стаж) и total_income (общий доход) есть строки с отсутсвующими данными.
Сразу видно, что значения в строке days_employed некорректны:
- во-первых есть отрицательные значения.
- во-вторых значение в 4 строке 340266. Исходя из описания данных, это общий трудовой стаж в днях, это означает что здесь указан трудовой стаж 932 года, что неможет соответсвовать действительности. 

### My Multi Word Header

### 2.1 Заполнение пропусков <a id='2'></a>

In [4]:
# для ознакомления с данными выведем столбцы со строками Nan
display(df[df['total_income'].isna()].head(3))

# посчитаем количество пропусков в total_income
print('Количество пропусков в total_income', len(df[df['total_income'].isna()]))

# посчитаем количество пропусков в days_employed
print('Количество пропусков в days_employed', len(df[df['days_employed'].isna()]))

# посчитаем отношение пропущенных данных к общему количеству строк
days_empty = len(df[df['days_employed'].isna()])
print(f'Отношение пропущенных данных к общему количеству строк {(days_empty / df.shape[0]):.2%}')


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости


Количество пропусков в total_income 2174
Количество пропусков в days_employed 2174
Отношение пропущенных данных к общему количеству строк 10.10%


### 2.2 выводы по пропущенным данным

Из приведенных данных видно:
- в столбцах days_employed и total_income есть строки с отсутсвующими данными.
- строки с пропущенными данными составляют 10%.
- т.к. пропущенные значения встречаются в двух столбцах одновременно, можно предположить, что эти данные не указывались умышленно, а все остальные данные указаны верно.
- при дальнейшей обработке данных можно удалить строки с пропущенными данными или заполнить их медианными значениями. Если просто удалить данные, то мы можем потерять сведения о 10% пользователей, что может существенно повлиять на результаты исследования, в данной ситуации лучше заполнить пропуски мединными значениями. При этом учитывая, что данные days_employed (трудовой стаж) имеют строки с некорректными данными и при этом сильно завышенными, брать средне-статистический показатель будет неправильным решением, т.к. некорректные данные сильно сместят среднестатистический показатель в большую сторону. 

Заполним пустые значения для total_income (ежемесячный доход).
Данные days_employed (трудовой стаж) заполним позже.

In [5]:
# расчет медианного значения для для total_income — ежемесячный доход
median_total_income = df['total_income'].median()
print('Медиальное значение для total_income (ежемесячный доход)', median_total_income)

# замена значения nan на 'median_total_income'
df['total_income'] = df['total_income'].fillna(value=median_total_income)

Медиальное значение для total_income (ежемесячный доход) 145017.93753253992


### 3. Проверка данных на аномалии и их исправление. <a id='3'></a>

В связи с тем, что в 4 строке были обнаружен некорректный трудовой стаж, для проверки остальных данных days_employed на слишком большие значения возьмем условную величину 85 лет - средняя максимальная продолжительность жизни в Японии. Прибавим возраст выхода на работу 18 лет, получаем, продолжительность жизни при таком показателе должны быть 103 года, можно сделать предположение, что таких данных не должно быть больше 1 %

In [6]:
# посчитаем количество отрицательных значений days_employed - трудовой стаж
days_negative = len(df[df['days_employed'] < 0])
print('Строки с отрицательным трудовым стажем', days_negative)

# посчитаем отношение отрицательных значений days_employed (трудовой стаж) к общему количеству строк
# поделим количество строк с отрицательным значением на общее количество строк
print(f'Отношение отрицательных значений days_employed (трудовой стаж) к общему количеству строк {(days_negative / df.shape[0]):.2%}')

# посчитаем количество значений dob_years - возраст клиента равное 0
dob_negative =  len(df[df['dob_years'] == 0]) #max(df['dob_years'])
print('Количество строк с возрастом клиента равным нулю', dob_negative)

# посчитаем отношение отрицательных значений dob_years (возраст клиента) к общему количеству строк
# поделим количество строк с отрицательным возрастом клиента на общее количество строк
print(f'Отношение строк равных нулю  dob_years (возраст клиента) к общему количеству строк {(dob_negative / df.shape[0]):.2%}')

# посчитаем количество отрицательных значений children - количество детей
children_negative = len(df[df['children'] < 0])
print('Строки с отрицательным количество детей', children_negative)

# посчитаем отношение отрицательных значений children (количество детей) к общему количеству строк
# поделим количество строк с отрицательным количеством детей на общее количество строк
print(f'Отношение отрицательных значений children (количество детей) к общему количеству строк {(children_negative / df.shape[0]):.2%}')


# посчитаем количество строк с трудовым стажем больше  85 лет
print ('Строки с трудовым стажем больше 85 лет', len(df[df['days_employed'] > 340025])) # 85 лет * 365 дней = 340025 дней

# посчитаем отношение значений  больших 85 лет к общему количеству строк
days_too_big = len(df[df['days_employed'] > 340025])
print(f'Отношение значений больших 85 лет к общему количеству строк {(days_too_big / df.shape[0]):.2%}')

Строки с отрицательным трудовым стажем 15906
Отношение отрицательных значений days_employed (трудовой стаж) к общему количеству строк 73.90%
Количество строк с возрастом клиента равным нулю 101
Отношение строк равных нулю  dob_years (возраст клиента) к общему количеству строк 0.47%
Строки с отрицательным количество детей 47
Отношение отрицательных значений children (количество детей) к общему количеству строк 0.22%
Строки с трудовым стажем больше 85 лет 2908
Отношение значений больших 85 лет к общему количеству строк 13.51%


Из приведенных данных можно сделать выводы:
- на данном этапе исследования необходимо обратиться за уточнением в данных, т.к. больше 73% данных с отрицательными значениями, больше 13% строк с неверными значениями, больше 10% строк с пропущенными значениями
- Данными с возрастом клиента 0 в данной работе прейдется пренебречь, т.к. таких данных 0.47%, можно предположить, что это некорректный ввод данных, либо ситуации когда клиент отказался укзывать свой возраст.
- можно предположить, что данные были введены некоректно либо был использован неверный формат, однако исходя из того, что в условии задания сказано, что данные могут иметь отрицательные значения, будем проводить исследование дальше.

Для дальнейших расчетов, заменим отрицательные значения в столбцах трудовой стаж и дети (days_employed, children). До уточнения данных будем исходить из предположения, что здесь ошибка в исходных данных.

In [7]:
# заменим отрицательные значения days_employed - трудовой стаж
df['days_employed'] = abs(df['days_employed'])

# заменим отрицательные значения children - дети
df['children'] = abs(df['children'])

# проверим количество отрицательных значений days_employed - трудовой стаж
print('Строки с отрицательным трудовым стажем после корректировки', len(df[df['days_employed'] < 0]))

# проверим количество отрицательных значений children - дети
print('Строки с отрицательным значением "дети" после корректировки', len(df[df['days_employed'] < 0]))


Строки с отрицательным трудовым стажем после корректировки 0
Строки с отрицательным значением "дети" после корректировки 0


### 3.1. Изменение типов данных.

In [8]:
# расчет медианного значения для для days_employed (трудовой стаж)
median_days_employed = df['days_employed'].median()
print('Медианное значения для для days_employed (трудовой стаж)', median_days_employed)
print()

# заменим значения nan на 'median_days_employed'
df['days_employed'] = df['days_employed'].fillna(value=median_days_employed)

#  проверим столбцы days_employed и total_income на пустые значения       
df.info()


Медианное значения для для days_employed (трудовой стаж) 2194.220566878695

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
# изменим вещественный тип данных для total_income — ежемесячный доход, на целочисленный
# с помощью метода astype()

df['total_income'] = df['total_income'].astype('int')

# проверим тип данных total_income после корректировки
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


### 3.2. Удаление дубликатов.

Перед удалением дубликатов переведем данные в нижний регистр.

In [10]:
# перед удалением дубликатов приведем данные в нижний регистр
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

# проверка размеров таблицы до удаления дубликатов
print('Размер таблицы до удаления дубликатов', df.shape)

# проверка размеров таблицы до удаления дубликато
df = df.drop_duplicates().reset_index(drop=True)

# проверка размеров таблицы после удаления дубликатов
print('Размер таблицы после удаления дубликатов', df.shape) 


Размер таблицы до удаления дубликатов (21525, 12)
Размер таблицы после удаления дубликатов (21454, 12)


В виде дубликатов был удален 71 явный дубликат, что составляет 0,3% можно предположить, что эти данные были продублированы в результате неккоректного сохранения базы данных или других ошибок.

Выведем уникальный список целей кредита. Для этого воспользуемся методом groupby() и count() их использование в данном случае предпочтительней метода value_counts() т.к. список сформируется сгруппированным по наименованию цели кредита. 

In [11]:
# для вывода так же можно воспользоватся методом count():
#purpose_unic = df['purpose'].value_counts()
#print(purpose_unic)

# выведем список уникальных значений методом groupby() и count():
purpose_unic = df.groupby('purpose')['purpose'].count()
print(purpose_unic)


purpose
автомобили                                478
автомобиль                                494
высшее образование                        452
дополнительное образование                460
жилье                                     646
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     471
на покупку подержанного автомобиля        478
на покупку своего автомобиля              505
на проведение свадьбы                     768
недвижимость                              633
образование                               447
операции с жильем                         652
операции с коммерческой недвижимостью     650
операции с недвижимостью                  675
операции со своей недвижимостью           627
покупка жилой недвижимости                606
покупка жилья                             646
покупка жилья для сдачи                   651
покупка жилья для семьи                   638
покупка коммерческой недви

Из приведенного списка сразу видно, что цели кредита слишком часто дублируют друг друга, это является поводом обратить внимание на ввод исходных данных менеджером или оператором.

### 4. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма. <a id='4'></a>

In [12]:
# Создадим два новых датафрейма со столбцами:

# df_education = education_id и education — в первом датафрэйме;
# df_family_status = family_status_id и family_status — во втором датафрэйме.

df_education = df[['education_id', 'education']]
df_education = df_education['education'].drop_duplicates().reset_index(drop=True)

df_family_status = df[['family_status_id', 'family_status']]
df_family_status = df_family_status['family_status'].drop_duplicates().reset_index(drop=True)

# выведем заголовки для проверки
display(df_education.head())
display(df_family_status.head())

# Удалим из исходного датафрейма столбцы education и family_status

df = df.drop(['education', 'family_status'], axis = 1)

# выведем заголовок для проверки
display(df.head())

0                 высшее
1                среднее
2    неоконченное высшее
3              начальное
4         ученая степень
Name: education, dtype: object

0          женат / замужем
1         гражданский брак
2           вдовец / вдова
3                в разводе
4    не женат / не замужем
Name: family_status, dtype: object

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,сыграть свадьбу


### 5. Категоризация дохода. <a id='5'></a>

In [13]:
# создадим функцию, income_group которая будет принимать новое значение на основе входящих данных
# новое значение - группа, входящие данные - размер дохода
def income_group(income_index):
    
    if income_index < 30001:
        return 'E'
    if income_index >= 30001 and income_index < 50000:
        return 'D'
    if income_index >= 50001 and income_index < 200000:
        return 'C'
    if income_index >= 200001 and income_index < 1000000:
        return 'B'
    return 'А' 

# создадим новый столбец в который поместим результат работы функции
# функция проходим по значения столбца 'total_income'
df['total_income_category'] = df['total_income'].apply(income_group)

# для проверки выведем заголовок измененной таблицы
display(df.head())
# для проверки посчитаем количество строк по каждой категории дохода
display(df['total_income_category'].value_counts())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C


C    16016
B     5041
D      350
А       25
E       22
Name: total_income_category, dtype: int64

### 6. Категоризация целей кредита. <a id='6'></a>

In [14]:
# создадим новый столбец, скопируем данные из purpose и проверим его содержимое
df['purpose_category'] = df['purpose']
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,покупка жилья,B,покупка жилья
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C,приобретение автомобиля
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,покупка жилья,C,покупка жилья
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,дополнительное образование,B,дополнительное образование
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C,сыграть свадьбу


По условиям задания создадим 4 группы "Цель кредита" на основании существующих 32.
Для этого предварительно в ручную отсортируем имеющиеся группы и разделим их на 4 группы:
- 'операции с автомобилем'
- 'операции с недвижимостью'
- 'проведение свадьбы'
- 'получение образования'

Далее создадим функцию, которая пройдет по имеющимся данным и на основании ручной выборки и заменит назвение имеющихся групп на новые.

In [15]:
#'операции с автомобилем'
# на вход функции подаются список из неправильных значений и строка с правильным значением
# минусом использования данной функции в данном случае является большая ручная выборка

def replace_wrong_purpose(wrong_values, correct_value):
# перебираем неправильные имена 
    for wrong_value in wrong_values:
        # и для каждого неправильного имени вызываем метод replace()                   
        df['purpose_category'] = df['purpose_category'].replace(wrong_value, correct_value)

# список неправильных имён 'операции с автомобилем'
duplicates_avto = ['автомобили', 'автомобиль', 'на покупку автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'приобретение автомобиля', 'свой автомобиль', 'сделка с автомобилем', 'сделка с подержанным автомобилем']                 
# правильное имя
name_avto = 'операции с автомобилем'
# вызов функции, replace()                                         
replace_wrong_purpose (duplicates_avto, name_avto)                 

# 'операции с недвижимостью'
duplicates_realty = ['операции с недвижимостью', 'операции со своей недвижимостью', 'недвижимость', 'жилье', 'операции с жильем', 'операции с коммерческой недвижимостью', 'покупка жилой недвижимости', 'покупка жилья', 'покупка жилья для сдачи', 'покупка жилья для семьи', 'покупка коммерческой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'ремонт жилью', 'строительство жилой недвижимости', 'строительство недвижимости', 'строительство собственной недвижимости']                 
name_realty = 'операции с недвижимостью'
replace_wrong_purpose (duplicates_realty, name_realty) 

# 'проведение свадьбы'
duplicates_marry = ['свадьба', 'сыграть свадьбу', 'на проведение свадьбы']                 
name_marry = 'проведение свадьбы'
replace_wrong_purpose (duplicates_marry, name_marry)

# 'получение образования'
duplicates_education = ['получение образования', 'образование', 'высшее образование', 'получение высшего образования', 'получение дополнительного образования', 'дополнительное образование', 'заняться высшим образованием', 'заняться образованием', 'профильное образование']                 
name_education = 'получение образования'
replace_wrong_purpose (duplicates_education, name_education)

# проверим данные, сделаем группировку данных по новой категории
display(df['purpose_category'].value_counts())

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Выводы по предобработке данных:

- для корректного проведения исследования необходимо обратиться за уточнением в данных, т.к. больше 73% данных с отрицательными значениями, больше 13% строк с неверными значениями, больше 10% строк с пропущенными значениями. Данными с возрастом клиента 0 в данной работе прейдется пренебречь, т.к. таких данных 0.47%, можно предположить, что это некорректный ввод данных, либо ситуации когда клиент отказался укзывать свой возраст.
- после корректировок исходных данных, итоговые значения поменялись, что может критично повлиять на итоговые выводы исследования.
    

### 7. Выводы. Ответы на вопросы. <a id='7'></a>

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [51]:
# узнаем количество должников
# сгруппируем данные по столбцу children и просуммируем значения по столбцу debt (его значение 0 или 1)
children_debt = df.groupby('children')['debt'].sum()

# узнаем сколько всего должников в каждой группе
children_count = df.groupby('children')['children'].count()

# расчитаем процент невозврата разделив количество невозврата на общее количество человек и умножим на 100
children_percent = ((df.groupby('children')['debt'].sum() / df.groupby('children')['children'].count()))*100
# print(children_debt, children_count, children_percent)

display(pd.concat([children_debt, children_count, children_percent], keys=['Debtor', 'Total', 'Percent'], axis=1))


,Debtor,Total,Percent
children,,,
0,1063,14091,7.543822
1,445,4855,9.165808
2,194,2052,9.454191
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000
20,8,76,10.526316


##### Вывод 1:
Из приведенной таблицы наглядно видно, что в целом количество должников без детей намного выше чем количество дожников с детьми, однако это вызвано тем, что общее количество кредитов выданное лицам без детей выше. 
В целом видно, что процент невозврата для заемщиков без детей ниже на 2%.
Нужно отметить, что эти данные являются достоверными, т.к. на результаты исследований не повлияли вносимые в данные изменения, проведенные в ходе анализа.
Необходимо отметить, что в таблице встречаются неккоректные данные - количество детей -1, таких строк 47 т.к. этих данных меньше 1% они не были учтены, так же строка с количеством детей 20 скорее всего предполагает количество детей больше 5. 

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
# узнаем количество должников по количеству детей
# сгруппируем данные по столбцу family_status и просуммируем значения по столбцу debt (его значение 0 или 1)
family_debt = df.groupby('family_status_id')['debt'].sum()

# узнаем сколько всего должников в каждой группе
family_count = family_debt + df.groupby('family_status_id')['family_status_id'].count()

# расчитаем процент невозврата разделив количество невозврата на общее количество человек и умножив на 100
family_percent = ((df.groupby('family_status_id')['debt'].sum() / df.groupby('family_status_id')['family_status_id'].count()))*100
# print(family_debt, family_count, family_percent)

# создадим сводную таблицу с полученными данными
data = [['в разводе', 85, 1195, 7.11],
       ['вдовец / вдова', 63, 959, 6.56],
       ['гражданский брак', 388, 4151, 9.34],
       ['женат / замужем', 931, 12339, 7.54],
        ['не женат / не замужем', 274, 2810, 9.75]]
        
columns = ['Семейный статус','Количество должников', 'Всего человек', '% должников']

research_family_result = pd.DataFrame(data=data,columns=columns)
display(research_family_result)

,Семейный статус,Количество должников,Всего человек,% должников
0,в разводе,85,1195,7.11
1,вдовец / вдова,63,959,6.56
2,гражданский брак,388,4151,9.34
3,женат / замужем,931,12339,7.54
4,не женат / не замужем,274,2810,9.75


##### Вывод 2:
Из приведенной таблицы наглядно видно, что есть небольшая корреляция в результатах: самый низкий процент невозврата для вдовцов, а самый высокий процент невозврата для тех кто не был в браке.

Надо отметить, что эти данные достоверные, т.к. изменения по данным показателям в исходные данные не вносились. 

##### Вопрос 3: Есть ли зависимость между доходом и возвратом кредита в срок?

In [35]:
# узнаем количество должников по каждой группе дохода
# сгруппируем данные по столбцу family_status и просуммируем значения по столбцу debt (его значение 0 или 1)
income_debt = df.groupby('total_income_category')['debt'].sum()

# узнаем сколько всего должников в каждой группе
income_count = df.groupby('total_income_category')['total_income_category'].count()

# расчитаем процент невозврата разделив количество невозврата на общее количество человек и умножив на 100
income_percent = ((df.groupby('total_income_category')['debt'].sum() / df.groupby('total_income_category')['total_income_category'].count()))*100
#print(income_debt, income_count, income_percent)

# создадим сводную таблицу с полученными данными
data = [['B: 200 000 - 1 000 000', 356, 5040, 7.06],
       ['С: 50 000 - 200 000', 1360, 16016, 8.49],
       ['D: 30 000 - 50 000', 21, 350, 6.00],
       ['E: от 0   - 30 000', 2, 22, 9.09],
        ['А: свыше 1 000 000', 2, 26, 7.69]]
        
columns = ['Группа','Количество должников', 'Всего человек', '% должников']

research_income_result = pd.DataFrame(data=data,columns=columns)
display(research_income_result)


,Группа,Количество должников,Всего человек,% должников
0,B: 200 000 - 1 000 000,356,5040,7.06
1,С: 50 000 - 200 000,1360,16016,8.49
2,D: 30 000 - 50 000,21,350,6.00
3,E: от 0 - 30 000,2,22,9.09
4,А: свыше 1 000 000,2,26,7.69


##### Вывод 3:
Для разных групп дохода видно, что самый низкий процент невозврата для группы D: а самый высокий процент для группы Е:

Надо отметить, эти данные могут быть не достоверные, т.к. до проведения группировки 10.10% были заполнены медиальными значениями, что могло существенно повлиять на результат.

In [19]:
# узнаем количество должников для каждой группы целей кредита
# сгруппируем данные по столбцу purpose_category и просуммируем значения по столбцу debt (его значение 0 или 1)
purpose_debt = df.groupby('purpose_category')['debt'].sum()

# узнаем сколько всего должников в каждой группе
purpose_count = df.groupby('purpose_category')['purpose_category'].count()

# расчитаем процент невозврата разделив количество невозврата на общее количество человек и умножив на 100
purpose_percent = ((df.groupby('purpose_category')['debt'].sum() / df.groupby('purpose_category')['purpose_category'].count()))*100


#print(purpose_debt, purpose_count, purpose_percent)

# создадим сводную таблицу с полученными данными
data = [['операции с автомобилем', 403, 4306, 9.35],
       ['операции с недвижимостью', 782, 10811, 7.23],
       ['получение образования', 370, 4013, 9.22],
       ['проведение свадьбы', 186, 2324, 8.00]]
        
        
columns = ['Цель кредита','Количество должников', 'Всего человек', '% должников']

research_purpose_result = pd.DataFrame(data=data,columns=columns)
display(research_purpose_result)


,Цель кредита,Количество должников,Всего человек,% должников
0,операции с автомобилем,403,4306,9.35
1,операции с недвижимостью,782,10811,7.23
2,получение образования,370,4013,9.22
3,проведение свадьбы,186,2324,8.00


##### Вывод 4:
По целям кредита есть небольшая корреляция: самый низкий % должников для операций с недвижимостью, самый высокий % должников для операций с автомобилем.

Надо отметить, эти данные достоверные, т.к. изменений в исходные данные для проведения анализа не вносилось.

## Общий вывод:
Сделаем сводную таблицу для обоснования общего вывода.

In [20]:
# создадим сводную таблицу по количеству детей
display(research_children_result)

# создадим сводную таблицу по семейному статусу
display(research_family_result)

,Количество детей,Количество должников,Всего человек,% должников
0,0,1063,14149,7.54
1,1,444,4818,9.21
2,2,194,2055,9.44
3,3,27,330,8.18
4,4,4,41,9.75
5,5,0,9,0.00
6,20,8,76,10.52


,Семейный статус,Количество должников,Всего человек,% должников
0,в разводе,85,1195,7.11
1,вдовец / вдова,63,959,6.56
2,гражданский брак,388,4151,9.34
3,женат / замужем,931,12339,7.54
4,не женат / не замужем,274,2810,9.75


Из полученных в результате анализ данных можно сделать вывод, что действительно есть взаимосвязь между количеством детей, семейным статусом и процентом прочек по кредиту.
- Минимальный показатель просроченной задолженности для вдовцов 6.56%
- Максимальный показатель просроченной задолженности для кредиторов с количеством детей больше 6 - 10.52%
- Разница между максимальным и минимальным процентом должников 3.96%


In [21]:
# создадим сводную таблицу с полученными данными
data = [['Min', '7.51 - нет детей', '6.56 - вдовец','7.06 - от 200 тыс до 1 млн', '7.23 - недвижимость'],
       ['Max', '10.52 - 20 детей', '7.75 - не женат', '9.09 - от 0 до 30 тыс', '9.35 - автомобиль']]

columns = ['Показатель группы', 'Количество детей','Семейный статус', 'Группа дохода', 'Цель кредита']

research_genres_result = pd.DataFrame(data=data,columns=columns)
display(research_genres_result)


,Показатель группы,Количество детей,Семейный статус,Группа дохода,Цель кредита
0,Min,7.51 - нет детей,6.56 - вдовец,7.06 - от 200 тыс до 1 млн,7.23 - недвижимость
1,Max,10.52 - 20 детей,7.75 - не женат,9.09 - от 0 до 30 тыс,9.35 - автомобиль


Из полученных в результате анализа данных можно сделать следующие выводы:
- минимальный процент невозврата 6.56 и 7.06 для вдовцов, а так же кредиторов с доходом от 200 тыс до 1 млн рублей.
- максимальный процент невозврата 10.52 и 9.35 для лиц с 6 и более детьми, а так же целью кредита "Операции с автомобилем".
- максимальный разброс в показателях по возврату кредита 3.96 процента.

Выявлен максимальный разброс в невозврате кредита в 3.96 % между разными показателями.

В анализе не использовались следующие данные: трудовой стаж, возраст, образование, пол, тип дохода.
Анализ проведенный после уточнения исходных данных, а так же с использованием указанных выше данных может выявить более значительные закономерности.